<a href="https://colab.research.google.com/github/srushtibhavsar/Gambling/blob/main/FinetuningLlama2OnHiwiData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need sentencepiece for the llama2 slow tokenizer
!pip install sentencepiece

# we need einops, used by falcon-7b, llama-2 etc
# einops (einsteinops) is used to simplify tensorops by making them readable
!pip install -q -U einops

# we need to install datasets for our training dataset
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
! pip install jsonlines


In [3]:
import json
import pandas as pd
from datasets import Dataset, DatasetDict
import jsonlines

In [4]:

# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "cleaned_dataset_dfs_edges_new.jsonl"

# Fine-tuned model name
new_model = "llama-2-7b-chuk-test"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 10

In [5]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)


In [6]:
# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #This parameter suggests that the model weights or parameters are loaded in a 4-bit format. In neural network quantization, 4-bit quantization typically represents a lower precision compared to standard 32-bit floating-point numbers.
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    #This parameter suggests that the model weights or parameters are loaded in a 4-bit format. In neural network quantization, 4-bit quantization typically represents a lower precision compared to standard 32-bit floating-point numbers.
    bnb_4bit_use_double_quant=True,
)


In [7]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [8]:
# don't use the cache
model.config.use_cache = False


In [9]:
# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [10]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
#prompt = "What does the REM keyword stand for in OPL?"
#prompt = "What was the first implementation of the OPL programming language?"
#prompt = "Who were involved in the creation of OPL for Psion?"
#prompt = "Did Colly Myers create OPL?"
prompt = "t # 1"
#prompt = "Write a limerick as comments in the OPL programming language. "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[INST] t # 1 [/INST]  I'm not sure I understand what you are saying with "t # 1". Could you explain?


In [11]:
# The instruction dataset to use
dataset_name = "srushtibhavsar/HiwiData"
from datasets import load_dataset

# Load the dataset
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
dataset

Dataset({
    features: ['text'],
    num_rows: 660
})

In [13]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [14]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
)


In [16]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
    per_device_train_batch_size=2,          # 4 seems reasonable
    gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=0,                           # we're not gonna save
    logging_steps=10,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=True,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=-1,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",           # constant seems better than cosine
    report_to="tensorboard"
)

In [17]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,                # use our lora peft config
    dataset_text_field="text",
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # use the llama tokenizer
    args=training_arguments,                # use the training arguments
    packing=False,                          # don't need packing
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:173: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/660 [00:00<?, ? examples/s]

In [18]:
# Train model
trainer.train()

{'loss': 1.5947, 'learning_rate': 4e-05, 'epoch': 0.06}
{'loss': 1.5562, 'learning_rate': 8e-05, 'epoch': 0.12}
{'loss': 1.7049, 'learning_rate': 0.00012, 'epoch': 0.18}
{'loss': 1.7643, 'learning_rate': 0.00015600000000000002, 'epoch': 0.24}
{'loss': 0.9835, 'learning_rate': 0.000196, 'epoch': 0.3}
{'loss': 0.5709, 'learning_rate': 0.00019998438638373762, 'epoch': 0.36}
{'loss': 0.4315, 'learning_rate': 0.0001999304196488835, 'epoch': 0.42}
{'loss': 0.5402, 'learning_rate': 0.00019983792783499226, 'epoch': 0.48}
{'loss': 0.4948, 'learning_rate': 0.0001997069465994188, 'epoch': 0.55}
{'loss': 0.3182, 'learning_rate': 0.00019953752643792272, 'epoch': 0.61}
{'loss': 0.3211, 'learning_rate': 0.00019932973266520142, 'epoch': 0.67}
{'loss': 0.455, 'learning_rate': 0.0001990836453897097, 'epoch': 0.73}
{'loss': 0.3693, 'learning_rate': 0.0001987993594827769, 'epoch': 0.79}
{'loss': 0.3155, 'learning_rate': 0.0001984769845420317, 'epoch': 0.85}
{'loss': 0.2901, 'learning_rate': 0.000198116644

TrainOutput(global_step=1650, training_loss=0.2545765022798018, metrics={'train_runtime': 1804.9023, 'train_samples_per_second': 3.657, 'train_steps_per_second': 0.914, 'train_loss': 0.2545765022798018, 'epoch': 10.0})